Import the libraries such as **OS** and **Pandas** to process the file path and files respectively.
An important library here is **Surprise** library. It is used here for reading the dataset, perform SVD and also for cross_validation operation.

# CMPE 256 - Programming assignment - Recommender systems
# Abhishek Singh (012566398)

In [1]:
import os
import pandas as pd
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise import Reader, Dataset, SVD, evaluate
from surprise.model_selection import cross_validate

**train** and **test** CSV  that are provided, are read below to the **df_trainfile** and **df_testfile** dataframes. df_trainfile contains the dataset with **tab seperated values** instead of commma as in typical csv file.

In [2]:
# provides teh path to train and test dataset file
df_trainfile = pd.read_csv('train.csv')
df_testfile = pd.read_csv('test.csv')

# converting train dataset from train.csv to tab seperated values train_tab.data
df_trainfile.to_csv('train_tab.data', sep='\t', index=False )

Fetching the path of the **tab-seperated** data file for further processing by **load_from_file** fuction. **df_test_rt** loads the copy of the test data subsequent **iterations**.

In [3]:
# getting the file path
# this is required by the load_from_file function 
file_path_tab = os.path.expanduser('train_tab.data')

# keeping a copy of test data
df_test_rt = df_testfile

**reader**-> reads the data from file, **sep**-> tells the **seperation** of the data or acts as **delimiter**, seprating two consequtive data blocks.

In [4]:
# reader: reads the data and gives label to each column
# sep specifies the type of delimeter 
# rating_scale: specifies the the scale for the prediction
data_reader = Reader(line_format='user item rating', sep='\t', rating_scale=(0, 100))

# loads the file content into the surprise library specific format
data = Dataset.load_from_file(file_path_tab, reader=data_reader)

# Retrieve the trainset.
trained_set = data.build_full_trainset()

Now we have to** build an algorithm** and train it as well. We will fist load the deafult **SVD** algorithm for that and then train it using the train the algorithm using **trained_set**

In [5]:
# use the algorithm with its default values
default_algorithm = SVD()

# train the model with trainset
default_algorithm.fit(trained_set)

Since the model is trained now, we will test the model using **Root Mean Sqaure Error(RMSE)** as evaluation metric.

In [6]:
evaluate(default_algorithm, data, measures=['RMSE'])

C:\Users\Abhishek\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\Abhishek\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 16.8412
------------
Fold 2
RMSE: 16.9417
------------
Fold 3
RMSE: 16.7209
------------
Fold 4
RMSE: 16.8589
------------
Fold 5
RMSE: 16.7955
------------
------------
Mean RMSE: 16.8316
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [16.841239373881827,
                             16.941715256642333,
                             16.72089720077727,
                             16.858859923724836,
                             16.795466306494983]})

*Create* an empty **list** to store the predicted values. Now the every record is *iterated* for the complete set of test data and *rating* is **predicted** for each **user-track** pair.

In [7]:
# list to store the predictions
predict_result = {}

# iterating row-wise over the test data to predict the ratings for each user-track pair 
for index, row in df_test_rt.iterrows():

    pred = default_algorithm.predict(str(row['User']), str(row['Track']), verbose=False)
    predict_result[index] = pred[3]

New data frame is **created** to load the load the ***result  file in .csv*** file. The new dataframe is formatted in the required format (joining by **hyphen**)

In [8]:
# to make additinal copy of dataframe for flexibilty in manipulation
df_final = df_test_rt

# creating a new dataframe to create a .csv result file
df_newdf = pd.DataFrame()


In [9]:
# to view the dataframe
df_final.head()

,User,Track
0,466,0
1,1173,0
2,566,1
3,49,3
4,772,3


In [10]:
# Concatenating the '-' as per the requirement
df_final['Id'] = df_final['User'].astype(str)+'-'+df_final['Track'].astype(str)

In [11]:
#  to view the dataframe
df_final.head()

,User,Track,Id
0,466,0,466-0
1,1173,0,1173-0
2,566,1,566-1
3,49,3,49-3
4,772,3,772-3


Ratings **column** is added to add the predicted **Ratings** alongwith combined **user-track** pair.

In [12]:
# adding a column for predicted Ratings alongside the combined user-track pair
df_newdf['Id'] = pd.Series(df_final['Id'], index=df_testfile.index, dtype=str)

df_newdf['Rating'] = pd.Series(predict_result, index=df_testfile.index, dtype=int)

In [13]:
# to view the dataframe
df_newdf.head()

,Id,Rating
0,466-0,51
1,1173-0,39
2,566-1,37
3,49-3,32
4,772-3,24


In [14]:
# save the dataframe to results.csv - as per the submission format 
df_newdf.to_csv('MySubmission_012566398_01.csv', index = False)